<a href="https://colab.research.google.com/github/shere-khan/machine_learning/blob/master/AirbnbInStoreCredit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Introduction/thoughts
How would we go about understanding if an in-store credit payment refund option is beneficial to the abnb platform?
The proposed solution was pretty unclear and ambiguous.
It was something  along the lines of modeling $y$ = "some kind of revenue dollar value" given a set of features, then testing to see if $y$ increased when we added a feature representing the new in-store payment option (categorical).
I realized somethings were patently wrong with the proposed solution.

## Proxy model approach
- $y$ = "revenue dollar value" is too ambiguous. When I think about what the label of training data points would look like, revenue doesn't make as much sense. I think we would have to narrow it down to something like "revenue generated by customer $x$ in $m$ day period"
- It might be better to use some kind of proxy metric to revenue. For instance, my initial suggestion was let $y$ be the probability of a user using the in-store credit or not. But that doesn't make sense becuase we need to be able to compare the two groups, with the new feature and without the new feature. If user's don't have the in-store credit option, modeling $y$ in this way doesn't make sense.
- Perhaps, $y$ = "probability of a user rebooking" is a better proxy.
- What if a user doesn't rebook but still uses the credit eventually. Did the credit encourage the customer to return? How would we measure that using a proxy? We can't measure if the user spent the credit directly.
- $y$ = "probability of user spending at least the original amount of money in $m$ days after cancellation". How do we define "rebooking"? Is it booking again within a certain time window? Is it spending the credit that was given? Perhaps, we can generally define rebooking as a guest spending at least the same amount of money as a canceled booking within a certain period of time from the cancellation.
- In a typical randomized controlled trial (RCT) scenario, say with testing medication. We assume users in group A and group B will both take the medication. In our case, treatment is giving the user an option to receive in-store credit, but the user might not necessarily choose this option. Do we remove all users that didn't choose the option? This reminds me of simulating a bandit offline using historical data. We only take as a reward the rows where the user actually selected the arm they were shown.




### Covariates
- Historical booking data
- Cancellation data
- Demographic data
- Content of messages exchanged between user and host. Or past message content from user
- <b>Covariates that best approximate a user that would select in-store credit as an option.</b> This is the most important yet hardest problem to solve for in this scenario where we're using a model that can approximate the in-store credit benefits. Basically, if we have no historical data that includes the in-store credit option, we have no way of including that as a feature. But perhaps we can use other features as signal for users that WOULD select the in-store credit option if they were given the change. For instance, say different credit cards have different refund latencies. Than including credit card type or latency days might be an indication for a user that would choose the in-store credit option.

### Training data
All cancellation events. A row should include a guest_id, host_id, listing_id. From there we can get customer/host/listing features. Should we include both user and host induced cancellations?

#### Modeling approach
Let's say $y$ = "probability of a customer rebooking". We could feed the data into a standard ML model and look to see if the probability of booking is higher for users <b>with</b> the proxy covariates than it is for users <b>without</b>.



Going through the above exercise made me ask whether this could be a use case for causal inference (CI) methods.
If we aren't running an AB experiment, how do we know what a user would have done if they had presented with the option to receive in-store credit.
We can't train a supervised model without labeled, historical data.
But CI techniques seem to only work if we have historical data that already has data containing the control and treatment groups.
So then where does the benefit of CI come into play here?

####<b>Perhaps we can augment the A/B testing process or somehow reuse data from AB tests in order to reduce the overhead and cost involved in running AB tests.</b>

## Causal approach: Propensity Score Matching (PSM)
We might be able to use psm in the following way.
Say we use an MAB or some method to assign users to control (no in-store credit option) and treatment (in-store credit option). 
We can use a regression model to estimate propensity scores for users. 
Then we can match users based on their propensity scores (literature suggests methods like KNN).
This allows us to group users that have similar observable characteristics but differ in their treatment status.
Then you can use methods to compare outcomes of interest between treatment and control groups.
Can measure the average treatment effect outcomes such as the likehood of a user to rebook as well as revenue generated by those users, and compare the treatment effects between groups.

Psm seems to work best for scenarios where we have existing RCT data where users are assigned to groups based on certain features.

####Questions
- What happens if we don't have historical data that uses treatment group in question?
- Can we look into using external data sets to provide an approximation?
- Can we use historical data sets that may been used for other experiments?
- What do we think using CI methods will buy us? One purpose for CI is to understand what the treatment effect would've been for patients if they had been assigned to different experimental groups. iow what would've happened if we had assigned a user to a different group. 
- this idea of incurring cost. So causal inference might allow us to get an estimate of performance without incurring the cost of running a "full" experiment.
- How would using propensity scores reduce the cost incurred by running a traditional multi armed bandit, would it reduce the amount of data we need to predict an outcome (revenue, prob of rebooking)?


### Alternatives to traditional AB testing/Areas to look into
1. AB testing with a multi-armed bandit as opposed to traditional frequentist approach. This could reduce cost and overhead because the traditional approach could overestimate things like sample size or the amount of time the experiment needs to run in order for power to be reached.
2. Performing causal analysis using historical data, whether from MAB or some kind of RCT. This might involve using external data to approximate features from treatment group